In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
import datetime
import math
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

from warnings import filterwarnings
import ccxt 

In [155]:
#แปลงข้อมูลจาก CCXT เป็น pandas DataFrame

ETH_data = ccxt.binance().fetch_ohlcv('ETH/USDT', timeframe='1d')
df = pd.DataFrame(ETH_data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Date'] = pd.to_datetime(df['Date'], unit='ms')
df['Date'] = df['Date'].dt.date
df.set_index('Date', inplace=True)

df.index[0]


datetime.date(2022, 10, 20)

In [118]:
def reshape_data(df, time_steps):
    
    X = []
    y = []

    for i in range(time_steps, len(df)):
        X.append(df[i-time_steps:i,0])
        y.append(df[i,0])
        
    return np.array(X), np.array(y)

In [162]:
def predict_values_for_future_dates(model, data, start_date, num_dates, time_steps):
    predictions = []

    current_date = datetime.combine(start_date, datetime.min.time())
    
    for _ in range(num_dates):
        input_data = data[-time_steps:].values
        input_data = input_data.reshape(1, time_steps, 1)
        
        prediction = model.predict(input_data)
        predictions.append(prediction[0, 0])
        
        current_date += timedelta(days=1)
        
        data = pd.concat([data, pd.DataFrame({'close': prediction[0, 0]}, index=[current_date])])

    return predictions

In [120]:
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
time_steps = 20
data = df.filter(['Close'])
dataset = data.values

training_data_len = math.ceil(len(dataset) * .8)
train_data = dataset[:training_data_len, :]
test_data = dataset[training_data_len-time_steps:, :]
train_dates = data[:training_data_len].index
test_dates = data[training_data_len-time_steps:].index
scaler = MinMaxScaler()

train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)
X_train, y_train = reshape_data(train_data, time_steps)
X_test, y_test = reshape_data(test_data, time_steps)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [121]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [122]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(380, 20, 1)
(100, 20, 1)
(380,)
(100,)


In [123]:
model = Sequential([
    LSTM(40, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.20),
    LSTM(40, return_sequences=False),
    Dropout(0.20),
    Dense(16),
    Dropout(0.20),
    Dense(1)
])

c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [124]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 20, 40)         │         6,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 20, 40)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 40)             │        12,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,353 (79.50 KB)

 Trainable params: 20,353 (79.50 KB)

 Non-trainable params: 0 (0.00 B)

In [125]:
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics = ['mean_absolute_error']
             )

In [126]:
history = model.fit(
    X_train,
    y_train,
    callbacks=early_stop,
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50


12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.2209 - mean_absolute_error: 0.4061
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0475 - mean_absolute_error: 0.1772
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0343 - mean_absolute_error: 0.1462
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0266 - mean_absolute_error: 0.1312
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0218 - mean_absolute_error: 0.1178
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0233 - mean_absolute_error: 0.1227
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0188 - mean_absolute_error: 0.1076
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0211 - mean_absolute_error: 0.1133
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0196 - mean_absolute_error: 0.1082
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0176 - mean_absolute_error: 0.1024 
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - lo

In [127]:
model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0362 - mean_absolute_error: 0.1101  


[0.06592820584774017, 0.13219359517097473]

In [128]:
losses = pd.DataFrame(history.history)
losses.head()

,loss,mean_absolute_error
0,0.133359,0.295642
1,0.038902,0.158654
2,0.030022,0.139059
3,0.024573,0.126138
4,0.020428,0.115653


In [129]:
y_pred_train = model.predict(X_train)
y_pred_train = scaler.inverse_transform(y_pred_train)

y_train_normal = y_train.reshape((y_train.shape[0], -1))
y_train_normal = scaler.inverse_transform(y_train_normal)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


In [130]:
y_pred_test = model.predict(X_test)
y_pred_test = scaler.inverse_transform(y_pred_test)

y_test_normal = y_test.reshape((y_test.shape[0], -1))
y_test_normal = scaler.inverse_transform(y_test_normal)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [131]:
last_results = pd.DataFrame({'close': data.values.reshape(-1, )}, index=data.index)

In [132]:
last_results['close'] = scaler.transform(last_results['close'].values.reshape(-1, 1))

In [133]:
data.iloc[-1]


Close    3420.5
Name: 2024-03-02, dtype: float64

In [161]:
start_date = data.index[-1]
num_dates = 500
type(start_date)

datetime.date

In [163]:
p = predict_values_for_future_dates(model, last_results, start_date, num_dates+1, time_steps)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

,Close
Date,
2022-10-20,1282.57
2022-10-21,1299.75
2022-10-22,1313.61
2022-10-23,1364.20
2022-10-24,1343.61
...,...
2024-02-27,3242.36
2024-02-28,3383.10
2024-02-29,3340.09


In [167]:
data.index[0]

datetime.date(2022, 10, 20)

In [169]:
NEW_DATES = [data.index[-1]]
for _ in range(num_dates):
    data_append = datetime.date(data.index[-1] + pd.DateOffset(days=_+1))
    NEW_DATES.append(data_append)




[datetime.date(2024, 3, 2),
 datetime.date(2024, 3, 3),
 datetime.date(2024, 3, 4),
 datetime.date(2024, 3, 5),
 datetime.date(2024, 3, 6),
 datetime.date(2024, 3, 7),
 datetime.date(2024, 3, 8),
 datetime.date(2024, 3, 9),
 datetime.date(2024, 3, 10),
 datetime.date(2024, 3, 11),
 datetime.date(2024, 3, 12),
 datetime.date(2024, 3, 13),
 datetime.date(2024, 3, 14),
 datetime.date(2024, 3, 15),
 datetime.date(2024, 3, 16),
 datetime.date(2024, 3, 17),
 datetime.date(2024, 3, 18),
 datetime.date(2024, 3, 19),
 datetime.date(2024, 3, 20),
 datetime.date(2024, 3, 21),
 datetime.date(2024, 3, 22),
 datetime.date(2024, 3, 23),
 datetime.date(2024, 3, 24),
 datetime.date(2024, 3, 25),
 datetime.date(2024, 3, 26),
 datetime.date(2024, 3, 27),
 datetime.date(2024, 3, 28),
 datetime.date(2024, 3, 29),
 datetime.date(2024, 3, 30),
 datetime.date(2024, 3, 31),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 4, 2),
 datetime.date(2024, 4, 3),
 datetime.date(2024, 4, 4),
 datetime.date(2024, 4, 5)

In [171]:
RESULTS = pd.DataFrame({'close': p[:]}, index=NEW_DATES)
RESULTS['close'] = scaler.inverse_transform(RESULTS[['close']])

In [172]:
model.save("eth_lstm_model.h5") # salvando o modelo
joblib.dump(scaler, 'eth_scaler.pkl')

['eth_scaler.pkl']

In [173]:
str(data.index[-1])

'2024-03-02'